In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple, defaultdict
from pathlib import Path
from enum import Enum
from os.path import splitext

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

# Results

In [3]:
system = "perlmutter"
version = "v0.5.1"
oldversion = "v0.5.0"

## Current

In [4]:
new = analyze.Analysis(f"results/{system}/")
print(new)

Analysis for Celeritas v0.5.0-66+9f4ba76bd on perlmutter


In [5]:
analyze.calc_geo_frac(new)

count      mean       std
problem                     geo     arch                           
atlas-tilecal               orange  cpu     4.0  0.739838  0.004416
                            vecgeom cpu     4.0  0.646757  0.015900
cms-hgcal                   orange  cpu     4.0  0.729142  0.022510
                            vecgeom cpu     4.0  0.648874  0.006112
cms2018                     vecgeom cpu     4.0  0.694109  0.004425
cms2018+field+msc           vecgeom cpu     4.0  0.799035  0.012237
                                    gpu     4.0  0.975084  0.000936
testem15                    orange  cpu     4.0  0.452204  0.000668
testem15+field              orange  cpu     4.0  0.540102  0.002441
                                    gpu     4.0  0.580064  0.003742
testem15+field+msc          orange  cpu     4.0  0.631925  0.014896
                            vecgeom cpu     4.0  0.586340  0.008982
testem3-composite+field+msc orange  cpu     4.0  0.770957  0.021845
                                    gpu     4.0  0.866244  0.002580
                            vecgeom cpu     4.0  0.754893  0.015422
                                    gpu     4.0  0.899822  0.000332
testem3-composite+msc       orange  cpu     4.0  0.740531  0.009691
                            vecgeom cpu     4.0  0.707368  0.005524
testem3-expanded+field+msc  orange  cpu     4.0  0.773493  0.018145
                            vecgeom cpu     4.0  0.757802  0.008342
testem3-flat                orange  cpu     4.0  0.673786  0.008155
                            vecgeom cpu     4.0  0.564600  0.021943
testem3-flat+field          orange  cpu     4.0  0.711532  0.010168
testem3-flat+field+msc      orange  cpu     4.0  0.782479  0.022830
                                    gpu     4.0  0.867958  0.003304
                            vecgeom cpu     4.0  0.759557  0.010480
                                    gpu     4.0  0.931375  0.001303
testem3-flat+msc            orange  cpu     4.0  0.751436  0.006158

## Previous

In [6]:
old = analyze.Analysis(f"results-old/{system}/")
print(old, f"({oldversion})")

Analysis for Celeritas v0.5.0-dev.222+33193b7f2 on perlmutter (v0.5.0)


In [7]:
ftab = analyze.make_failure_table(old.failures())
ftab.to_frame()

,Failure


# Analysis

## Compare manually

In [19]:
def get_throughput(analysis):
    col = analysis.result['avg_event_per_time']
    return col.xs('vecgeom', level='geo').unstack('arch')

def get_slots(analysis):
    col = analysis.input['num_track_slots']
    return col.xs('vecgeom', level='geo').unstack('arch')

def compare(func):
    return 

In [22]:
get_throughput(new) / get_throughput(old) - 1

arch                                       cpu    cpu+g4       gpu    gpu+g4  \
problem                     instance                                           
atlas-tilecal               0        -0.394953  0.003041 -0.044679  0.282126   
                            1        -0.400524  0.000088 -0.046422  0.279727   
                            2        -0.396896 -0.013201 -0.048575  0.271690   
                            3        -0.418273  0.003622 -0.049623  0.263174   
cms-hgcal                   0        -0.332316  0.034221 -0.025306  0.301586   
                            1        -0.344703  0.031886 -0.026230  0.315164   
                            2        -0.333422  0.000537 -0.028709  0.320760   
                            3        -0.333555  0.006087 -0.027231  0.306930   
cms2018+field+msc           0        -0.235427 -0.066730 -0.019336  1.998694   
                            1        -0.193855 -0.052122 -0.037959  2.079291   
                            2        -0.203617 -0.049475 -0.038628  2.023586   
                            3        -0.206178 -0.017616 -0.038818  2.048397   
cms2018                     0        -0.423404 -0.065421 -0.107250  0.382504   
                            1        -0.420653 -0.038170 -0.104166  0.382095   
                            2        -0.404716 -0.100255 -0.108218  0.387237   
                            3        -0.418164 -0.056405 -0.105804  0.389884   
testem15+field+msc          0        -0.284986 -0.015402  0.022763  0.164745   
                            1        -0.296401 -0.034992  0.022852  0.168074   
                            2        -0.317892  0.032344  0.006581  0.205858   
                            3        -0.295933 -0.009646  0.018564  0.221908   
testem15+field              0              NaN       NaN       NaN       NaN   
                            1              NaN       NaN       NaN       NaN   
                            2              NaN       NaN       NaN       NaN   
                            3              NaN       NaN       NaN       NaN   
testem3-composite+field+msc 0        -0.137864  0.012740  0.001638  0.966524   
                            1        -0.098270 -0.013259 -0.008220  0.952655   
                            2        -0.173837 -0.049993 -0.002114  0.972260   
                            3        -0.106605 -0.020956  0.001192  0.953743   
testem3-composite+msc       0        -0.215660 -0.043107 -0.006139  0.340269   
                            1        -0.178468 -0.038798 -0.004822  0.386608   
                            2        -0.170865 -0.046098 -0.002867  0.341149   
                            3        -0.214332 -0.042409 -0.004640  0.343474   
testem3-expanded+field+msc  0        -0.118571 -0.003213  0.001084  0.948978   
                            1        -0.131644 -0.017729 -0.009769  0.938065   
                            2        -0.133145 -0.012107 -0.006338  0.944984   
                            3        -0.141017 -0.006871  0.000473  0.955821   
testem3-flat+field+msc      0        -0.151696 -0.019336  0.002829  1.053366   
                            1        -0.165919 -0.015619  0.001331  1.044966   
                            2        -0.145030  0.002821 -0.003402  1.037445   
                            3        -0.185116 -0.042127 -0.000327  1.053446   
testem3-flat                0        -0.448280  0.037532 -0.004140  0.441387   
                            1        -0.396700 -0.009817  0.003495  0.403694   
                            2        -0.447638  0.019040 -0.000798  0.402904   
                            3        -0.466701  0.014791 -0.002949  0.402663   

arch                                  gpu+sync  
problem                     instance            
atlas-tilecal               0              NaN  
                            1              NaN  
                            2              NaN  
                            3              NaN  
cms-hgcal                

In [23]:
get_slots(new) / get_slots(old)

arch,cpu,cpu+g4,gpu,gpu+g4,gpu+sync
problem,,,,,
atlas-tilecal,16.0,1.0,1.0,4.0,NaN
cms-hgcal,16.0,1.0,1.0,4.0,NaN
cms2018+field+msc,16.0,1.0,1.0,4.0,1.0
cms2018,16.0,1.0,1.0,4.0,NaN
testem15+field+msc,16.0,1.0,1.0,4.0,NaN
testem15+field,NaN,NaN,NaN,NaN,1.0
testem3-composite+field+msc,16.0,1.0,1.0,4.0,1.0
testem3-composite+msc,16.0,1.0,1.0,4.0,NaN
testem3-expanded+field+msc,16.0,1.0,1.0,4.0,NaN


## Performance drop between versions

In [87]:
#analyses = {a.version: a for a in [frontier, frontier_v050]}
analyses = {
    oldversion: old,
    version: new,
}
throughput = {k: a.result['avg_event_per_time'] for k, a in analyses.items()}
plot_rel_to = old

In [88]:
def plot_ratio(result_dict):
    rel = analyze.summarize_instances(result_dict[version] / result_dict[oldversion])
    rel.dropna(inplace=True)
    fig, ax = plt.subplots(layout="constrained")
    plot_rel_to.plot_results(ax, rel)
    ax.grid(which='both')
    ax.legend()
    ax.text(0.98, 0.02,
        f"{version} / {oldversion} - 1 on {system}",
        va='bottom', ha='right',
        fontstyle='italic', color=(0.5,)*3, size='xx-small',
        transform=ax.transAxes,
        zorder=-100
    )

    return ax

In [89]:
steps = {k: a.result['avg_steps_per_primary'] for k, a in analyses.items()}

In [90]:
ax = plot_ratio(throughput)
ax.set_ylabel(f"Throughput [events/sec]")
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-throughput.pdf", transparent=True)
plt.close()

In [91]:
ax = plot_ratio(steps)
ax.set_ylabel(f"Work [steps/primary]")
ax.get_figure().savefig(f"results-old/{system}/rel-work.png", dpi=300)
ax.get_figure().savefig(f"results-old/{system}/rel-work.pdf", transparent=True)
plt.close()